## Capstone 1 
# San Francisco Bay Water Quality

ref. [Water quality of SF Bay home page](https://sfbay.wr.usgs.gov/access/wqdata/index.html)
     
     

## Unit 5 - Data Wrangling, Stations

### Tasks

The first step in completing your capstone project is to collect data. Depending on your dataset, you may apply some of the data wrangling techniques that you learned in this unit.

Include answers to these questions in your submission:
   * What kind of cleaning steps did you perform?

   * How did you deal with missing values, if any?

   * Were there outliers, and how did you handle them?


## Data Acquisition



### Station Location Information

#### Access

Location data for "standard" stations is available from [ScienceBase](https://www.sciencebase.gov/catalog/item/5966abe6e4b0d1f9f05cf551).

However, more complete location data is available in tables at [sfbay.wr.usgs.gov](https://sfbay.wr.usgs.gov/access/wqdata/overview/wherewhen/where.html). These tables include the genral location of each station (by geographical landmark) as well as data for "non-standard" stations which are sampled less often.

These tableswere copied, pasted into a spreadsheet, then exported as CSV. Header fields were edied o remove newlines and several fields were modified to remove artifacts before exporting to CSV format.
#### Files

   1. `SFBayStationLocations.csv`


#### Data Format

The Station Locations file is CSV format with one header row and 5 columns.

<small> 
```
Station Number, General Location, North Latitude, West Longitude, Depth MLW (meters)
```
</small>




## Setup

Import libraries

In [19]:
# Import useful libraries

import pandas as pd
import matplotlib.pyplot as plt
import datetime
import re
import json


## Read in the Stations Tables

In [20]:
# Read in Station locations
st_df = pd.read_csv('Data/orig/SFBayStationLocationsTable.csv', 
                    dtype={'Station Number' : str},
                    header=0)


In [21]:
st_df.columns

Index(['Station Number', 'General Location', 'North Latitude',
       'West Longitude', 'Depth MLW (m)'],
      dtype='object')

In [22]:
st_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
Station Number      51 non-null object
General Location    48 non-null object
North Latitude      51 non-null object
West Longitude      51 non-null object
Depth MLW (m)       44 non-null float64
dtypes: float64(1), object(4)
memory usage: 2.1+ KB


In [25]:
# rename 'Station Number' as 'Station'
st_df.rename(columns={"Station Number": "Station"}, inplace=True)

In [26]:
# Create a list of stations
station_list_tmp = st_df['Station'].tolist()
print(*station_list_tmp) 

657 649 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25 26 27 28 29 29.5 30 31 32 33 34 35 36 662 659 655 654 653 652 651 650 411 407 405 12.5 19 28.5


In [29]:
# The stations, in order from the Sacramento River south to San Jose
station_list = ['662', '659', '657', '655', '654', '653', '652', '651', '650', '649', 
                '2', '3', '4', '407', '411','5', '6', '7', '405', '8', '9', '10', 
                '11', '12','12.5', '13', '14', '15', '16', '17', '18', '19', '20', 
                '21', '22', '23', '24', '25', '26','27', '28', '28.5', '29', '29.5', 
                '30', '31', '32', '33', '34', '35', '36']

#sort the stations


st_df.Station = pd.Categorical(st_df.Station, 
                      categories=station_list,
                      ordered=True)

st_df.sort_values('Station', inplace=True)
st_df.reset_index(drop=True, inplace=True)

In [30]:
st_df

,Station,General Location,North Latitude,West Longitude,Depth MLW (m)
0,662,Prospect Island,38 13.6',-121 40.2',10.1
1,659,Old Sac. River,10.7',40.0',10.1
2,657,Rio Vista,38 9.1',-121 41.3',10.1
3,655,N.of Three Mile Slough,7.3',42.1',10.1
4,654,NaN,6.3',42.5',NaN
5,653,Mid-Decker Island,6.3',43.2',10.1
6,652,Towland's Landing,5.2',44.8',NaN
7,651,NaN,4.7',45.8',NaN
8,650,NaN,4.3',46.5',NaN
9,649,Sacramento River,3.6',48.0',10.1


## Many records in the table do not include geographic degrees (they inherit from the station on the row above).

I want to fill in this data.

In [31]:
# First, extract the degrees from the curent Lat and Long columns
st_df['North Lat Degrees'] = st_df['North Latitude'].str.extract('^(3[78]) ')
st_df['West Long Degrees'] = st_df['West Longitude'].str.extract('^(-12[12]) ')

In [32]:
# Then fill forward for rows that did not specify degrees
st_df['North Lat Degrees'].fillna(method='ffill', inplace=True)
st_df['West Long Degrees'].fillna(method='ffill', inplace=True)

In [33]:
# Next, remove the degrees from the columns that contain minutes
st_df['North Lat Minutes'] = st_df['North Latitude'].str.extract('(\d{1,2}\.\d)\'$')
st_df['West Long Minutes'] = st_df['West Longitude'].str.extract('(\d{1,2}\.\d)\'$')

In [34]:
st_df.head(20)

,Station,General Location,North Latitude,West Longitude,Depth MLW (m),North Lat Degrees,West Long Degrees,North Lat Minutes,West Long Minutes
0,662,Prospect Island,38 13.6',-121 40.2',10.1,38,-121,13.6,40.2
1,659,Old Sac. River,10.7',40.0',10.1,38,-121,10.7,40.0
2,657,Rio Vista,38 9.1',-121 41.3',10.1,38,-121,9.1,41.3
3,655,N.of Three Mile Slough,7.3',42.1',10.1,38,-121,7.3,42.1
4,654,NaN,6.3',42.5',NaN,38,-121,6.3,42.5
5,653,Mid-Decker Island,6.3',43.2',10.1,38,-121,6.3,43.2
6,652,Towland's Landing,5.2',44.8',NaN,38,-121,5.2,44.8
7,651,NaN,4.7',45.8',NaN,38,-121,4.7,45.8
8,650,NaN,4.3',46.5',NaN,38,-121,4.3,46.5
9,649,Sacramento River,3.6',48.0',10.1,38,-121,3.6,48.0


Now I can drop the original Latitude and Longitude columns

In [35]:
st_df.drop(columns=['North Latitude', 'West Longitude', 
                    ], inplace=True)

And replace them with decimal versions

In [36]:
st_df = st_df.astype({'North Lat Degrees': 'float',
              'North Lat Minutes': 'float',
              'West Long Degrees': 'float',
              'West Long Minutes': 'float'
             })

st_df['Latitude'] = st_df['North Lat Degrees'] + (st_df['North Lat Minutes']/60)
st_df['Longitude'] = abs(st_df['West Long Degrees']) + (st_df['West Long Minutes']/60)
st_df['Longitude'] = st_df['Longitude'] * (-1)

In [37]:
st_df

,Station,General Location,Depth MLW (m),North Lat Degrees,West Long Degrees,North Lat Minutes,West Long Minutes,Latitude,Longitude
0,662,Prospect Island,10.1,38.0,-121.0,13.6,40.2,38.226667,-121.670000
1,659,Old Sac. River,10.1,38.0,-121.0,10.7,40.0,38.178333,-121.666667
2,657,Rio Vista,10.1,38.0,-121.0,9.1,41.3,38.151667,-121.688333
3,655,N.of Three Mile Slough,10.1,38.0,-121.0,7.3,42.1,38.121667,-121.701667
4,654,NaN,NaN,38.0,-121.0,6.3,42.5,38.105000,-121.708333
5,653,Mid-Decker Island,10.1,38.0,-121.0,6.3,43.2,38.105000,-121.720000
6,652,Towland's Landing,NaN,38.0,-121.0,5.2,44.8,38.086667,-121.746667
7,651,NaN,NaN,38.0,-121.0,4.7,45.8,38.078333,-121.763333
8,650,NaN,NaN,38.0,-121.0,4.3,46.5,38.071667,-121.775000
9,649,Sacramento River,10.1,38.0,-121.0,3.6,48.0,38.060000,-121.800000


And rearrange the columns

In [38]:
st_df.columns

Index(['Station', 'General Location', 'Depth MLW (m)', 'North Lat Degrees',
       'West Long Degrees', 'North Lat Minutes', 'West Long Minutes',
       'Latitude', 'Longitude'],
      dtype='object')

In [39]:
st_df = st_df[['Station', 'General Location', 
              'Latitude', 'Longitude',
              'North Lat Degrees', 'North Lat Minutes',
              'West Long Degrees','West Long Minutes', 
              'Depth MLW (m)'
             ]]

In [40]:
st_df.head()

,Station,General Location,Latitude,Longitude,North Lat Degrees,North Lat Minutes,West Long Degrees,West Long Minutes,Depth MLW (m)
0,662,Prospect Island,38.226667,-121.670000,38.0,13.6,-121.0,40.2,10.1
1,659,Old Sac. River,38.178333,-121.666667,38.0,10.7,-121.0,40.0,10.1
2,657,Rio Vista,38.151667,-121.688333,38.0,9.1,-121.0,41.3,10.1
3,655,N.of Three Mile Slough,38.121667,-121.701667,38.0,7.3,-121.0,42.1,10.1
4,654,NaN,38.105000,-121.708333,38.0,6.3,-121.0,42.5,NaN


In [41]:
#Save the station locations file
st_df.to_csv('Data/SFBayStationLocations.csv', index=False)

<hr style="border: 5px solid green;">

Next time, we can read the data in with

```
st_df = pd.read_csv('Data/SFBayStationLocations.csv', 
                    header=0,
                    dtype={'Station Number' : str}
                    )

```
